# STN Training Module

In [1]:
import os
os.chdir('genre_classification_289a/src')

In [2]:
import torch
import torch.nn as nn
import os
from model import STN, MLP
import torch.optim as optim

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
from features import get_data_loaders, FramedFeatureDataset, FeatureDataset, DatasetSettings

### DCNN Training

In [5]:
#STN targets
agfs = ['subgenres'] #'subgenre', 'mfcc'
genre = False #False if not genre STN

target = None

if (genre and len(agfs) > 0) or (len(agfs) > 1):
    raise 'Error: can only train one STN target at a time'
elif genre:
    target = 'genre'
elif len(agfs) == 1:
    target = agfs[0]
else:
    raise 'Error: could not parse training target'

print('Training STN target: {}'.format(target))
    
#dataset
dataset_name = 'fma_small'

Training STN target: subgenres


In [6]:
settings = DatasetSettings(dataset_name, 'fma_metadata')
dataset = FramedFeatureDataset(settings,  agfs=agfs, genre=genre)
print("Num genres: ", settings.num_genres)
print(settings.genre_counts)

Num genres:  8
Pop              1000
International    1000
Instrumental     1000
Folk             1000
Rock              999
Experimental      999
Electronic        999
Hip-Hop           997
Name: genre_top, dtype: int64


In [7]:
AGF_DIM = 40
output_size = None
if target == 'genre':
    output_size = settings.num_genres
elif target in ['subgenres', 'mfcc']:
    output_size = AGF_DIM
else:
    raise 'Error: invalid target'
    
stn = STN(output_size)
stn.to(device)
stn = nn.DataParallel(stn)

In [8]:
## Training Parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(stn.parameters(), lr=0.001)
epochs = 30
batch_size = 64
valid_split = 0.2

In [9]:
trainloader, validloader = get_data_loaders(dataset, batch_size, valid_split)

In [10]:
import torch.nn.functional as F
from sklearn.metrics import f1_score

def validate(stn, label_name):
    
    with torch.no_grad():
        stn.eval()
        
        all_pred = []
        all_true = []
        
        for i, data in enumerate(validloader, 0):
            inputs, labels = data[0].to(device), data[1][label_name].to(device)
            
            out = stn(inputs)
            loss = F.cross_entropy(out, labels)
            
            all_pred.append(out.argmax(dim=1))
            all_true.append(labels)
            
        all_pred = torch.cat(all_pred)
        all_true = torch.cat(all_true)
        
        curr_f1 = f1_score(all_true.cpu(), all_pred.cpu(), average='micro')
        
        print('Validation f1 score: {}'.format(curr_f1))

In [ ]:
#Train it
for epoch in range(epochs):  # loop over the dataset multiple times
    
    print('Starting epoch', epoch + 1)
    
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data[0].to(device), data[1][target].to(device)  #data[1]['{argument for agf being trained}']
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = stn(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        

        # print statistics
        running_loss += loss.item()
        if i % 50 == 49:    # print every 30 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 30))
            running_loss = 0.0
            
    if epoch % 5 == 4:
        validate(stn, 'genre')
        stn.train()

print('Finished Training')

Starting epoch 1


### Save Model

In [ ]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
torch.save(stn, model_file)

## Load & Eval Model

In [ ]:
model_file = '../models/DCNN_{}_{}'.format(dataset_name, target)
model = torch.load(model_file)
model

In [ ]:
validate(model, target)

## Previous STN Val Accuracies
FMA_SMALL
* genre: 89.X% (can't reload model to validate b/c didn't set a random seed during training and the random split of training / validation data is different so we're mixing training data into the validation set and getting  unrealistically high validation accuracies)
* mfcc: (prev. incorrect val: 97.36%)

FMA_MED
* genre: 89.3%
